In [39]:
import sqlalchemy
import pandas as pd

In [43]:
engine = sqlalchemy.create_engine('mssql+pyodbc://wbmserccar101/POVANA14')

## Adrien's things

In [45]:
#pd.read_sql_table('data', engine)
emissionsbyhousehold = pd.read_sql_table("emissionsbyhousehold",engine,schema="combined")
PerCapitaExpenditures= pd.read_sql_table("PerCapitaExpenditures",engine,schema="combined")
HHLDDATA= pd.read_sql_table("HHLDDATA",engine,schema="combined")
tradedatagtp= pd.read_sql_table("tradedatagtp",engine,schema="combined")
PERSDATA= pd.read_sql_table("PERSDATA",engine,schema="combined")
DecileComposite= pd.read_sql_table("DecileComposite",engine,schema="combined")

In [1]:
import os
os.makedirs("csvdata",exist_ok=True)

In [46]:
emissionsbyhousehold .to_csv("csvdata/emissionsbyhousehold.csv")
PerCapitaExpenditures.to_csv("csvdata/PerCapitaExpenditures.csv")
HHLDDATA             .to_csv("csvdata/HHLDDATA.csv"             )
tradedatagtp         .to_csv("csvdata/tradedatagtp.csv"         )
PERSDATA             .to_csv("csvdata/PERSDATA.csv"             )
DecileComposite      .to_csv("csvdata/DecileComposite.csv"      )